In [ ]:
import pandas as pd
from bokeh.charts import output_notebook, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.layouts import row
from bokeh.plotting import figure

In [ ]:
dataset = pd.read_csv("data.csv",index_col=0,parse_dates=True)


rich_weekday = lambda c: -1 if c["holiday"] else c["weekday"]

dataset.workingday = dataset.workingday==1
dataset.holiday = dataset.holiday==1
dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["season_s"] = dataset.season.map({1:"Printemps",2:"Ete",3:"Automne",4:"Hiver"})
dataset["weather_s"] = dataset.weather.map({1:"clear",2:"cloudy",3:"light rain",4:"crappy"})
dataset["rich_weekday"] = dataset.apply( rich_weekday, axis=1)

In [ ]:
gb =dataset.groupby(dataset.index.weekday)["count"].mean()
b = Bar(gb, values="count")
output_notebook() 
#output_file("bar.html")
show(b)

In [ ]:
#dataset.hour = dataset.index.hour.map(dict( (i,"%i->%i" % (4*(i/4),4*(i/4)+3)) for i in range(24) ))


gg =  dataset.groupby(["weekday","hour"])["count"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="count",color="weekday")

show(b)

In [ ]:
#dataset.hour = dataset.index.hour.map(dict( (i,"%i->%i" % (4*(i/4),4*(i/4)+3)) for i in range(24) ))

rich_weekday = lambda c: -1 if c["holiday"] else c["weekday"]

dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["rich_weekday"] = dataset.apply( rich_weekday, axis=1)
gg =  dataset.groupby(["weekday","hour"])["registered"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="registered",color="weekday")

show(b)

In [ ]:
#dataset["casu_biker"] = dataset["count"] - dataset["registered"] 

gg =  dataset.groupby(["weekday","hour"])["casual"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="casual",color="weekday")
show(b)

In [ ]:

gg =  dataset.groupby(["weekday","hour"])["registered"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="registered",color="weekday")
show(b)

In [ ]:
check = dataset.registered +dataset.casual - dataset["count"]
check.value_counts()

In [ ]:
h_ds = dataset.groupby( ["workingday","hour"])["count"].mean()

f = Bar(dataset,label="hour",values="count",agg="mean",group="holiday")
show(f)

In [ ]:
f = Bar(dataset,label="season_s",values="count",agg="mean",group="holiday")
show(f)

In [ ]:
f_casu = Bar(dataset,title = "casu as f(weather)",label="season_s",values="casual",agg="mean",group="weather_s")
f_reg = Bar(dataset,title = "registered as f(weather)",label="season_s",values="registered",agg="mean",group="weather_s")
show(row(f_casu,f_reg))

In [ ]:
f = BoxPlot(dataset,title = "registered as f(weather)",label="season_s",values="count")
show(f)

In [ ]:
#dataset[dataset.season==1]
dataset[dataset.weather==4]

In [ ]:
dataset["humidity_b"] = pd.cut(dataset.humidity,10)
f_casu = Bar(dataset,title="casu",label="humidity_b",values="casual",agg="mean",group="season_s")
f_reg = Bar(dataset,title="registered",label="humidity_b",values="registered",agg="mean",group="season_s")
show(row(f_casu,f_reg))

In [ ]:
dataset["temp_b"] = pd.cut(dataset.temp,10)
f_casu = Bar(dataset,title="casu",label="temp_b",values="casual",agg="mean",group="season_s")
f_reg = Bar(dataset,title="registered",label="temp_b",values="registered",agg="mean",group="season_s")

show(row(f_casu,f_reg))

In [ ]:
from bokeh.palettes import Spectral6
from bokeh.models import ColumnDataSource

grouped= dataset.groupby(["temp_b","humidity_b"])["count"].mean().reset_index()
grouped["temp"]=grouped.temp_b.str.extract("\((?P<temp>\d+\.\d+).*").apply(float)
grouped["humidity"]=grouped.humidity_b.str.extract("\((?P<temp>-?\d+(?:\.\d+)?).*")
grouped
#.apply(int)

mean_cnt =grouped["count"].mean()
grouped["sz"] = grouped["count"]/mean_cnt*15
colormap = dict( (i,Spectral6[i]) for i in range(len(Spectral6)))
cmap = pd.cut(grouped["count"],len(Spectral6),labels=Spectral6).tolist()
grouped["color_lgd"] = pd.cut(grouped["count"],len(Spectral6))

source = ColumnDataSource(dict(
    x=grouped.temp.tolist(),
    y=grouped.humidity.tolist(),
    color=cmap,
    sz=grouped["sz"].tolist(),
    label=grouped.color_lgd.tolist()
))
#.map(colormap)

#grouped
f = figure(plot_width=600, plot_height=600)
f.circle(x="x",y="y",size="sz", color="color",legend="label",source=source)
f.yaxis.axis_label = "Humidity"
f.xaxis.axis_label = "Temp (°C)"


show(f)

In [ ]:
dataset["windspeed_g"] = pd.cut(dataset.windspeed,10)

b = Bar(dataset,label="windspeed_g",values="count",agg="mean",group="season_s")
show(b)

In [ ]:
dset

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
label = dset["count"] 
del dset["count"], dset["registered"], dset["casual"]
dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["h_wd"] = dataset.weekday*100+dataset.hour

#dset_scaled = preprocessing.scale(dset)
pca = PCA(n_components="mle")

dt = DecisionTreeRegressor(max_depth=10)
clf = AdaBoostRegressor(n_estimators=1000,)
estimators = [('reduce_dim', pca), ('clf',clf)]
pipe = Pipeline(estimators)
scores = cross_val_score(clf, dset, label,cv=20,verbose=1,scoring="neg_mean_absolute_error",n_jobs=6)                          
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
clf = DecisionTreeRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV
clf.get_params()
param_grid = [{'learning_rate':[1,10],'n_estimators':[10,100,500,1000,5000]}]
gscv = GridSearchCV(
    estimator = clf,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",
    cv=5,
    verbose=2,
    n_jobs=6
)

f = gscv.fit(dset_scaled, label)

print f.best_estimator_
print f.best_score_

In [ ]:
from sklearn import svm
from sklearn import preprocessing
dset_scaled = preprocessing.scale(dset)
clf = svm.SVR(kernel='rbf',C=1,verbose=True, cache_size=1000)
#clf = svm.SVR(C=1e3,verbose=True)
scores = cross_val_score(clf, dset, label,cv=10,verbose=3,scoring="neg_median_absolute_error",n_jobs=6)
scores.mean()  
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
cross_val_score?

In [ ]:
dataset["count"].describe()